# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f8ad5037220>
├── 'a' --> tensor([[ 0.0617,  0.3741,  0.9139],
│                   [ 0.5760, -0.3701,  0.5074]])
└── 'x' --> <FastTreeValue 0x7f8ad50374c0>
    └── 'c' --> tensor([[ 2.7675, -0.7212,  0.4946, -0.7720],
                        [-0.4516,  0.4064, -0.7955, -0.4616],
                        [ 1.2276, -2.4756, -0.3230,  1.9856]])

In [4]:
t.a

tensor([[ 0.0617,  0.3741,  0.9139],
        [ 0.5760, -0.3701,  0.5074]])

In [5]:
%timeit t.a

75.9 ns ± 1.48 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8ad5037220>
├── 'a' --> tensor([[-0.2907, -0.7748, -0.8628],
│                   [ 1.3418, -0.5140,  0.5162]])
└── 'x' --> <FastTreeValue 0x7f8ad50374c0>
    └── 'c' --> tensor([[ 2.7675, -0.7212,  0.4946, -0.7720],
                        [-0.4516,  0.4064, -0.7955, -0.4616],
                        [ 1.2276, -2.4756, -0.3230,  1.9856]])

In [7]:
%timeit t.a = new_value

72.8 ns ± 1.7 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0617,  0.3741,  0.9139],
               [ 0.5760, -0.3701,  0.5074]]),
    x: Batch(
           c: tensor([[ 2.7675, -0.7212,  0.4946, -0.7720],
                      [-0.4516,  0.4064, -0.7955, -0.4616],
                      [ 1.2276, -2.4756, -0.3230,  1.9856]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0617,  0.3741,  0.9139],
        [ 0.5760, -0.3701,  0.5074]])

In [11]:
%timeit b.a

68.4 ns ± 1.11 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.9432,  1.7892, -0.8365],
               [ 0.2143, -0.2392,  1.4979]]),
    x: Batch(
           c: tensor([[ 2.7675, -0.7212,  0.4946, -0.7720],
                      [-0.4516,  0.4064, -0.7955, -0.4616],
                      [ 1.2276, -2.4756, -0.3230,  1.9856]]),
       ),
)

In [13]:
%timeit b.a = new_value

623 ns ± 11.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.05 µs ± 24.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.1 µs ± 178 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

264 µs ± 7.24 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

257 µs ± 11.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f8a1b858b50>
├── 'a' --> tensor([[[ 0.0617,  0.3741,  0.9139],
│                    [ 0.5760, -0.3701,  0.5074]],
│           
│                   [[ 0.0617,  0.3741,  0.9139],
│                    [ 0.5760, -0.3701,  0.5074]],
│           
│                   [[ 0.0617,  0.3741,  0.9139],
│                    [ 0.5760, -0.3701,  0.5074]],
│           
│                   [[ 0.0617,  0.3741,  0.9139],
│                    [ 0.5760, -0.3701,  0.5074]],
│           
│                   [[ 0.0617,  0.3741,  0.9139],
│                    [ 0.5760, -0.3701,  0.5074]],
│           
│                   [[ 0.0617,  0.3741,  0.9139],
│                    [ 0.5760, -0.3701,  0.5074]],
│           
│                   [[ 0.0617,  0.3741,  0.9139],
│                    [ 0.5760, -0.3701,  0.5074]],
│           
│                   [[ 0.0617,  0.3741,  0.9139],
│                    [ 0.5760, -0.3701,  0.5074]]])
└── 'x' --> <FastTreeValue 0x7f8a21efd400>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

42 µs ± 816 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f8a1b24f370>
├── 'a' --> tensor([[ 0.0617,  0.3741,  0.9139],
│                   [ 0.5760, -0.3701,  0.5074],
│                   [ 0.0617,  0.3741,  0.9139],
│                   [ 0.5760, -0.3701,  0.5074],
│                   [ 0.0617,  0.3741,  0.9139],
│                   [ 0.5760, -0.3701,  0.5074],
│                   [ 0.0617,  0.3741,  0.9139],
│                   [ 0.5760, -0.3701,  0.5074],
│                   [ 0.0617,  0.3741,  0.9139],
│                   [ 0.5760, -0.3701,  0.5074],
│                   [ 0.0617,  0.3741,  0.9139],
│                   [ 0.5760, -0.3701,  0.5074],
│                   [ 0.0617,  0.3741,  0.9139],
│                   [ 0.5760, -0.3701,  0.5074],
│                   [ 0.0617,  0.3741,  0.9139],
│                   [ 0.5760, -0.3701,  0.5074]])
└── 'x' --> <FastTreeValue 0x7f8ad0fc5ac0>
    └── 'c' --> tensor([[ 2.7675, -0.7212,  0.4946, -0.7720],
                        [-0.4516,  0.4064, -0.7955, -0.4616],
                 

In [23]:
%timeit t_cat(trees)

38.6 µs ± 1.7 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

76 µs ± 1.91 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 2.7675, -0.7212,  0.4946, -0.7720],
                       [-0.4516,  0.4064, -0.7955, -0.4616],
                       [ 1.2276, -2.4756, -0.3230,  1.9856]],
              
                      [[ 2.7675, -0.7212,  0.4946, -0.7720],
                       [-0.4516,  0.4064, -0.7955, -0.4616],
                       [ 1.2276, -2.4756, -0.3230,  1.9856]],
              
                      [[ 2.7675, -0.7212,  0.4946, -0.7720],
                       [-0.4516,  0.4064, -0.7955, -0.4616],
                       [ 1.2276, -2.4756, -0.3230,  1.9856]],
              
                      [[ 2.7675, -0.7212,  0.4946, -0.7720],
                       [-0.4516,  0.4064, -0.7955, -0.4616],
                       [ 1.2276, -2.4756, -0.3230,  1.9856]],
              
                      [[ 2.7675, -0.7212,  0.4946, -0.7720],
                       [-0.4516,  0.4064, -0.7955, -0.4616],
                       [ 1.2276, -2.4756, -0.3230,  1.9856]],

In [26]:
%timeit Batch.stack(batches)

96.6 µs ± 1.21 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 2.7675, -0.7212,  0.4946, -0.7720],
                      [-0.4516,  0.4064, -0.7955, -0.4616],
                      [ 1.2276, -2.4756, -0.3230,  1.9856],
                      [ 2.7675, -0.7212,  0.4946, -0.7720],
                      [-0.4516,  0.4064, -0.7955, -0.4616],
                      [ 1.2276, -2.4756, -0.3230,  1.9856],
                      [ 2.7675, -0.7212,  0.4946, -0.7720],
                      [-0.4516,  0.4064, -0.7955, -0.4616],
                      [ 1.2276, -2.4756, -0.3230,  1.9856],
                      [ 2.7675, -0.7212,  0.4946, -0.7720],
                      [-0.4516,  0.4064, -0.7955, -0.4616],
                      [ 1.2276, -2.4756, -0.3230,  1.9856],
                      [ 2.7675, -0.7212,  0.4946, -0.7720],
                      [-0.4516,  0.4064, -0.7955, -0.4616],
                      [ 1.2276, -2.4756, -0.3230,  1.9856],
                      [ 2.7675, -0.7212,  0.4946, -0.7720],
                   

In [28]:
%timeit Batch.cat(batches)

177 µs ± 3.62 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

529 µs ± 18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
